# COMP541  Spring 2018 Visualization Lab

* In this Lab you will implement some methods and experiment with several visualization techniques.<br/>
* Please read all the instructions and comments carefully.
* **IMPORTANT: NEXT STEP MAY TAKE A LITTLE BIT LONGER. PLEASE WAIT UNTILL ALL YOUR PACKAGES are installed.**

In [ ]:
# You may comment out that cell after installing all the necessary packages
for p in ("Knet","ArgParse","Images","Plots","Plotly","MAT")
    Pkg.installed(p) == nothing && Pkg.add(p)
end
# PLEASE WAIT TO FINISH PACKAGE INSTALLATION

In [ ]:
# A bit of setup as usual
using Knet, ArgParse,Images,Plots,MAT
const modelurl = "http://www.vlfeat.org/matconvnet/models/imagenet-resnet-101-dag.mat"
const imgurls = [("https://github.com/BVLC/caffe/raw/master/examples/images/cat.jpg", 284),
                 ("http://home.mweb.co.za/pa/pak04857/uniweb/animalimages/elephantthumb.jpg",387),
                 ("http://farm1.static.flickr.com/64/168461914_afe4852372.jpg",386),
                 ("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT_zYiGPRndbvGTKqff5PVp7gvOO1SWGprhnI9rcDFxmXEAx9SIgg", 228),
                ]
plotlyjs()

In [ ]:
# Takes a url or file name and returns an Array{RGB} 
# It does not resize or modify the original image. 
function loadimage(img)
    global _imgcache
    if contains(img,"://")
        info("Downloading $img")
        a0 = load(download(img))
    else
        a0 = load(img)
    end
    new_size = ntuple(i->div(size(a0,i)*224,minimum(size(a0))),2)
    a1 = Images.imresize(a0, new_size)
    i1 = div(size(a1,1)-224,2)
    j1 = div(size(a1,2)-224,2)
    b10 = a1[i1+1:i1+224,j1+1:j1+224]
    return b10
end


In [ ]:
# It takes output of loadimage and 
# modifies it by resizing and subtracting average image 
# provided my resnet file 
function prepare_img(img,averageImage)
    # ad-hoc solution for Mac-OS image
    macfix = convert(Array{FixedPointNumbers.Normed{UInt8,8},3}, channelview(img))
    c1 = permutedims(macfix, (3,2,1))
    d1 = convert(Array{Float32}, c1)
    e1 = reshape(d1[:,:,1:3], (224,224,3,1))     
    f1 = (255 * e1 .- averageImage) 
    g1 = permutedims(f1, [2,1,3,4])
    return g1
end

# Occlusion Experiments
You are going to occlude input image from different regions and <br\>
observe the probability of predicted class of occluded image.

For Occlusion Experiments, you need to:
* Complete ```pertubeimg``` method.
* Update the ```results``` matrix with the probability of the target class you get with the occluded image. Target class is the class the model predicts for the original image.<br\>
  

**```pertubeimg``` method**

It is used to obtain occluded versions of the orjinal image.
You are going to modify the input image by moving occlusion box through the image. You will return ```occluded_images``` array which stores  modified images, each of which has is  a ```KnetArray{Float32,4}``` array. The color of the occlusion box will be the mean color of original image.


Parameters of ```pertubeimg``` method: <br\>
* ```img```    : output of ```loadimage``` method. It's type is Array{RGB}. 
* ```avgimg``` : Average pixel values for each channel of RGB mapping. Obtained from the model file in main function.
* ```atype```  : KnetArray{Float32} or Array{Float32}
* ```psize```  : (w,h) tuple defining the size of occlusion box. 

**Hint**: You can use ```mean``` function to find the mean pixel values of image <br\>
**Hint-2**: Double check that you are occluding only a single region at a time in the input image <br\> 
**Hint-3**: You may want to use ```prepare_img``` method to convert your occluded image to desired type before pushing it to ```occluded_images``` .<br\> 

**Updating ```result``` matrix** <br\>
Inside the main function, you will classify each occluded image stored in ```occluded_images``` array. 
After each classification, you will store the probability of target class in the results matrix at the correct position. ```results``` is an 2D array, where each point corresponds to a occlusion region in the original image. 

In [ ]:
# Used for occlusion visualization. It takes 
# original image (output of loadimage) as
function pertubeimg(img,avgimg,atype;psize=(8,8))
    w,h = size(img)
    occluded_images = Any[]
    # we use these values in our occlusion box 
    meanvals = mean(img)
    occbox  = zeros(RGB{Float64},psize) .+ meanvals
    for c1=1:psize[1]:w
        occluded_images_cols = Any[]
        for c2=1:psize[2]:h
            # ANSWERS
            image=copy(img)
            if c1>w-psize[1] && c2>h-psize[2]
                image[c1:w,c2:h]=occbox
                elseif c1>w-psize[1] && c2<=h-psize[2]
                image[c1:w,c2:c2+psize[2]-1]=occbox
                elseif c1<=w-psize[1] && c2>h-psize[2]
                image[c1:c1+psize[1]-1,c2:h]=occbox
                else 
                image[c1:c1+psize[1]-1,c2:c2+psize[2]-1]=occbox
            end
            image1=prepare_img(image,avgimg)
            occluded_image=convert(atype,image1)
            push!(occluded_images_cols,occluded_image)            
            # ANSWERE
        end
        push!(occluded_images,occluded_images_cols)
    end
    return occluded_images
end


# Saliency Maps
We will compute saliency maps described in section 3.1 of [1].<br\>

First, you need to compute the **gradient of the image** with respect to the
unnormalized class score, not with respect to the normalized class probablitity.
To implement ```compute_saliency``` you need to understand and choose "correct" parts from ```classify``` function given above.

Hint: After implementing forward calculation Knet's ```grad``` function takes care gradients.

[1] Karen Simonyan, Andrea Vedaldi, and Andrew Zisserman. "Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps", ICLR Workshop 2014.

In [ ]:
# Computes the forward calculations of saliency
function compute_saliency(input_image, model_weights, gold_label, ms, f)
    #ANSWERS
    y1 = f(model_weights,input_image,ms)
    return y1[gold_label]
    #ANSWERE
end

In [ ]:
grad_saliency = grad(compute_saliency);#YOUR CODE HERE

In [ ]:
# Do not touch that function
mnistview(x,i)=colorview(Gray,permutedims(x[:,:,1,i],(2,1)))
function visualize_saliency(gimg1)
    g1 = abs.(gimg1);
    g2 = maximum(Array(g1), 3)
    g2[g2 .> 1.1f-3] += 0.1
    display(mnistview(g2,1))
end

# Weight Visualization
We will visualize the weights of specific layer's of Resnet101 model. <br\>

To do that, the only thing you need to to is complete ```weightvis``` method. <br\>
For the assignment, you need to visualize the first layer of the Resnet. <br\>
You can check the layers of Resnet101 model by looking the utility methods <br\>
defined at the bottom of this notebook. 
Parameters of the ```weightvis```:
* ```w```    : Weight arrays. 
* ```ms```   : Resnet meta parameters required for batchnorm
* ```mode``` : 1 is test mode 0 is train mode
* ```layernum```: ID of target layer where you visualize the weigths. Do not need to change it
* ```scale```: resizing scale used while visualizing weights. For example, if the scale is (4,4) then,  
    the target weight matrix,wx, will be resized 4*size(wx,1) x 4*size(wx,2)<br\>
    **Hint**: You need to shift all the  weight values between 0 and 1.<br\>
    **Hint-2**: You may want to look at RGB type and colorview method defined in Images package.<br\>
    **Hint-3**: To resize the image, you may want to lookat imresize method efined in Images package.

In [ ]:
function weightvis(w,ms;mode=1,layernum=1,scale=(4,4))
    # 1) Convert your weight to Array{Float32} type
    # 2) Clamp your elements in weight array so that all of them will be greater than or equal to zero
    # 3) If you have N^2 filters in that layer, you will display them in a NxN grid. You will store each
    # filter in a result array. 
    # 4) You will resize each filter by using scale parameter. You may want to use imresize method. 
    IJulia.clear_output(true)
    result = Any[]

    sqrtsize =  Int(floor(sqrt(size(w[layernum],4))))

    # ANSWERS
    w1=convert(Array,w[layernum])
    x=size(w1,1)
    y=size(w1,2)
    if size(w1,4)==sqrtsize^2
        for i=1:sqrtsize
            result_col = Any[]
            for j=1:sqrtsize
                R=reshape(w1[:,:,1,i*j],(x,y))
                R=(R.-minimum(R))./(maximum(R)-minimum(R))
                G=reshape(w1[:,:,2,i*j],(x,y))
                G=(G.-minimum(G))./(maximum(G)-minimum(G))
                B=reshape(w1[:,:,3,i*j],(x,y))
                B=(B.-minimum(B))./(maximum(B)-minimum(B))
                a=colorview(RGB,R,G,B)
                push!(result_col,Images.imresize(a,(scale[1]*x,scale[2]*y)))
                

            end
            push!(result,hcat(result_col...))
        end
    else
        println("nonsquare size")
    end
    # ANSWERE
    display(vcat(result...))
   # result1=[]
    #for i in 1:sqrtsize:length((result...))
     #   b=hcat(result[i:i+sqrtsize-1])
      #  push!(result1,b)
    #end
    #a=vcat(result1)
    #display(vcat(result1...))
end

In [ ]:
function main(args)
    s = ArgParseSettings()
    s.description = "Comp541, Spring 2018 Visualization Lab"
    @add_arg_table s begin
        ("--image";arg_type=Int; default=2; help="Image file or URL chosen from the imgurls array")
        ("--model"; default="imagenet-resnet-101-dag"; help="resnet model name")
        ("--top"; default=5; arg_type=Int; help="Display the top N classes")
        ("--atype"; default=(gpu()>=0 ? "KnetArray{Float32}" : "Array{Float32}"); help="array and float type to use")
        ("--task"; default="occlusion"; help="type of visualization. Options: occlusion|weightvis|saliency")
    end

    isa(args, AbstractString) && (args=split(args))
    o = parse_args(args, s; as_symbols=true)
    o[:image] = imgurls[o[:image]]
    println("opts=",[(k,v) for (k,v) in o]...)
    atype = eval(parse(o[:atype]))
    model = load_model(o[:model])
    avgimg = model["meta"]["normalization"]["averageImage"]
    avgimg = convert(Array{Float32}, avgimg)
    description = model["meta"]["classes"]["description"]
    w, ms = get_params(model["params"], atype)
    # get model by length of parameters
    modeldict = Dict(
        314 => (resnet101, "resnet101"))
    !haskey(modeldict, length(w)) && error("wrong resnet MAT file")
    resnet, name = modeldict[length(w)]
    
    # Occlusion Experiments
    if o[:task] == "occlusion"
        oimg  = loadimage(o[:image][1]) # You need 1 for imgurl
        oimg2 = convert(atype,prepare_img(oimg,avgimg))
        # predictions with corresponding probabilities for original image 
        p1,s1 = classify(w,resnet,oimg2,ms)
        # name of the class with highest probability
        pgold,cname = p1[s1[1:o[:top]]][1],description[s1[1:o[:top]]][1]
        println("top-1 class & probability of original image:")
        println(pgold,"\t",cname)
        # create perturbed images 
        inputs = pertubeimg(oimg,avgimg,atype;psize=(8,8))
        # store the probability of predicted class of perturbed image. 
        result = zeros(length(inputs),length(inputs[1]))
       

        println("----------------------------")
        println("top-1 class of perturbed images:")
        for i=1:length(inputs)
            for j=1:length(inputs[i])
                # classify each perturbed image 
                p2,s2 = classify(w,resnet,inputs[i][j],ms)
                # display the top class probability for perturbed img
                output = hcat(p2[s2[1:o[:top]]], description[s2[1:o[:top]]]);
                #println("i:$i, j:$j:",output[1,:]);flush(STDOUT)                
                  
                # ANSWERS
                result[i,j]=p2[s1[1]]
                #ANSWERE
            end
        end
    
    println("----------------------------")
    println("result matrix:")
    flush(STDOUT)
    display(result)
    result1=zeros(size(result))
    for i in 1:size(result)[1]
            result1[end-i+1,:]=result[i,:]
    end
    
    # plot heatmap and original image side by side 
    plot(plot(oimg,colorbar=false),heatmap(result1,c=:gray),size=(448,224))                    

    elseif o[:task] == "saliency"
        oimg = loadimage(o[:image][1])
        gold_label = o[:image][2] # we need to know the correct label to calculate the loss w.r.t to it
        oimg2 = convert(atype,prepare_img(oimg,avgimg))
        gradients_of_image = grad_saliency(oimg2, w, gold_label, ms, resnet)
        display(oimg)
        visualize_saliency(gradients_of_image)
    elseif o[:task] == "weightvis"
        weightvis(w,ms)
    else
        error("You've chosen unimplemented task")
    end
end

# Utility Functions 
Below, we add the utility functions needed through the lab. We encourage you to understand them.
<br\>Here is a brief list of these functions:
* Method for loading pretrained ```resnet101``` model 
* Methods for resnet layers and batchnormalization 
* Method that returns loaded weights and batchnorm parameters
* Most importantnly ```classify``` method, you can use directly to get a prediction for an image. 
* You may want to look at ```get_params``` and ```resnet``` methods for weight visualization subtask of the Lab.

In [ ]:
# RESNET building functions 
# Batch Normalization Layer
# works both for convolutional and fully connected layers
# mode, 0=>train, 1=>test

function classify(w,resnet,img,ms)
    y1 = resnet(w,img,ms)
    z1 = vec(Array(y1))
    s1 = sortperm(z1,rev=true)
    p1 = exp.(logp(z1))
    return p1,s1
end

function bnorm(w, x, ms; mode=1, epsilon=1e-5)
    mu, sigma = nothing, nothing
    if mode == 0
        d = ndims(x) == 4 ? (1,2,4) : (2,)
        s = prod(size(x,d...))
        mu = sum(x,d) / s
        x0 = x .- mu
        x1 = x0 .* x0
        sigma = sqrt(epsilon + (sum(x1, d)) / s)
    elseif mode == 1
        mu = shift!(ms)
        sigma = shift!(ms)
    end

    # we need getval in backpropagation
    push!(ms, getval(mu), getval(sigma))
    xhat = (x.-mu) ./ sigma
    return w[1] .* xhat .+ w[2]
end

function reslayerx0(w,x,ms; padding=0, stride=1, mode=1)
    b  = conv4(w[1],x; padding=padding, stride=stride)
    bx = bnorm(w[2:3],b,ms; mode=mode)
end

function reslayerx1(w,x,ms; padding=0, stride=1, mode=1)
    relu.(reslayerx0(w,x,ms; padding=padding, stride=stride, mode=mode))
end

function reslayerx2(w,x,ms; pads=[0,1,0], strides=[1,1,1], mode=1)
    ba = reslayerx1(w[1:3],x,ms; padding=pads[1], stride=strides[1], mode=mode)
    bb = reslayerx1(w[4:6],ba,ms; padding=pads[2], stride=strides[2], mode=mode)
    bc = reslayerx0(w[7:9],bb,ms; padding=pads[3], stride=strides[3], mode=mode)
end

function reslayerx3(w,x,ms; pads=[0,0,1,0], strides=[2,2,1,1], mode=1) # 12
    a = reslayerx0(w[1:3],x,ms; stride=strides[1], padding=pads[1], mode=mode)
    b = reslayerx2(w[4:12],x,ms; strides=strides[2:4], pads=pads[2:4], mode=mode)
    relu.(a .+ b)
end

function reslayerx4(w,x,ms; pads=[0,1,0], strides=[1,1,1], mode=1)
    relu.(x .+ reslayerx2(w,x,ms; pads=pads, strides=strides, mode=mode))
end

function reslayerx5(w,x,ms; strides=[2,2,1,1], mode=1)
    x = reslayerx3(w[1:12],x,ms; strides=strides, mode=mode)
    for k = 13:9:length(w)
        x = reslayerx4(w[k:k+8],x,ms; mode=mode)
    end
    return x
end

# mode, 0=>train, 1=>test
function resnet101(w,x,ms; mode=1)
    # layer 1
    conv1 = reslayerx1(w[1:3],x,ms; padding=3, stride=2, mode=mode)
    pool1 = pool(conv1; window=3, stride=2)

    # layer 2,3,4,5
    r2 = reslayerx5(w[4:33], pool1, ms; strides=[1,1,1,1], mode=mode)
    r3 = reslayerx5(w[34:72], r2, ms; mode=mode)
    r4 = reslayerx5(w[73:282], r3, ms; mode=mode)
    r5 = reslayerx5(w[283:312], r4, ms; mode=mode)

    # fully connected layer
    pool5  = pool(r5; stride=1, window=7, mode=2)
    fc1000 = w[313] * mat(pool5) .+ w[314]
end

_mcnurl = "http://www.vlfeat.org/matconvnet/models"
_mcndir = Pkg.dir("Knet","data","imagenet")

function load_model(name)
    global _mcncache
    if !isdefined(:_mcncache); _mcncache=Dict(); end
    if !haskey(_mcncache,name)
        matfile = "$name.mat"
        info("Loading $matfile...")
        path = joinpath(_mcndir,matfile)
        if !isfile(path)
            println("Should I download $matfile?")
            readline()[1] == 'y' || error(:ok)
            isdir(_mcndir) || mkpath(_mcndir)
            download("$_mcnurl/$matfile",path)
        end
        _mcncache[name] = matread(path)
    end
    return _mcncache[name]
end

function get_params(params, atype)
    len = length(params["value"])
    ws, ms = [], []
    for k = 1:len
        name = params["name"][k]
        value = convert(Array{Float32}, params["value"][k])

        if endswith(name, "moments")
            push!(ms, reshape(value[:,1], (1,1,size(value,1),1)))
            push!(ms, reshape(value[:,2], (1,1,size(value,1),1)))
        elseif startswith(name, "bn")
            push!(ws, reshape(value, (1,1,length(value),1)))
        elseif startswith(name, "fc") && endswith(name, "filter")
            push!(ws, transpose(reshape(value,size(value,3,4))))
        elseif startswith(name, "conv") && endswith(name, "bias")
            push!(ws, reshape(value, (1,1,length(value),1)))
        else
            push!(ws, value)
        end
    end
    map(wi->convert(atype, wi), ws),
    map(mi->convert(atype, mi), ms)
end


# Test Your Code 
You can use the one line below to test your implementations.
If you encounter with a text box asking for user input, 
please type 'y'. It will download pretrained resnet model file.

In [ ]:
TASK="weightvis"  # other alternatives: weightvis or saliency occlusion
main("--task weightvis --image 2")# Second argument is an image index to choose from imgurls array